In [1]:
import tensorflow as tf
tf.enable_eager_execution()

2024-05-07 07:10:00.534886: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import os
import random

# Function to load images and corresponding labels
def load_images_and_labels(folder_path):
    images = []
    labels = []
    label_names = []

    # Iterate over folders (labels)
    for label_folder in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label_folder)

        # Skip if it's not a directory
        if not os.path.isdir(label_path):
            continue

        # Append label name to the list
        label_names.append(label_folder)

        # Load images from the label folder
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            # Append image path to the list
            images.append(image_path)
            # Append label index to the labels list
            labels.append(len(label_names) - 1)  # Index of label name

    return images, labels, label_names

# Load images and labels
vggface2_folder = "Downloads/val"
vggface2_folder1="Downloads/test"
images, labels, label_names = load_images_and_labels(vggface2_folder)
images1,labels1,label_names1=load_images_and_labels(vggface2_folder1)
images=images+images1
labels=labels+labels1
label_names=label_names1
# Split data into train and test sets (adjust the split ratio as needed)
split_ratio = 0.95
num_images = len(images)
num_train = int(num_images * split_ratio)
random.seed(42)
train_indices = random.sample(range(num_images), num_train)

test_indices = list(set(range(num_images)) - set(train_indices))

# Write image paths and labels into train.txt and test.txt files
def write_to_file(file_path, image_paths, image_labels):
    with open(file_path, 'w') as file:
        for path, label in zip(image_paths, image_labels):
            file.write(f"{path} {label}\n")

write_to_file("Downloads/train.txt", [images[i] for i in train_indices], [labels[i] for i in train_indices])
write_to_file("Downloads/test.txt", [images[i] for i in test_indices], [labels[i] for i in test_indices])

print("Files train.txt and test.txt generated successfully.")


Files train.txt and test.txt generated successfully.


In [3]:

weight_decay = 1e-4
epoch_num = 300
hash_bit = 512
num_class = len(set(labels))
input_shape = (224, 224, 3)
label_smoothing = 0.1

# training config
batch_size = 128
train_num = len(train_indices) #6149
print(train_num)
iterations_per_epoch = int(train_num / batch_size) + 1
warm_iterations = iterations_per_epoch
print(iterations_per_epoch)


initial_learning_rate = 0.05
minimum_learning_rate = 0.0001

# test config
test_batch_size = 128
test_num = len(test_indices)
print(test_num)
test_iterations = int(test_num / test_batch_size) + 1

log_file = 'result/vggface2/densenet-512-4.18.txt'

# lr
learning_rate = [0.1, 0.01, 0.001]
boundaries = [80 * iterations_per_epoch, 120 * iterations_per_epoch]


# path
train_data_path = ''
test_data_path = ''
imagenet_train_path = 'Downloads/train.txt'
imagenet_test_path = 'Downloads/test.txt'

flowers17_train_path = "./flowers17/train.txt"
flowers17_test_path = "./flowers17/test.txt"

flowers102_train_path = "./oxford-102-flowers/train.txt"
flowers102_test_path = "./oxford-102-flowers/test.txt"

sun397_train_path = "../dataset/imagenet/train_label.txt"
sun397_test_path = "../dataset/imagenet/validation_label.txt"

short_side_scale = (256, 384)
aspect_ratio_scale = (0.8, 1.25)
hue_delta = (-36, 36)
saturation_scale = (0.6, 1.4)
brightness_scale = (0.6, 1.4)
pca_std = 0.1

mean = [103.939, 116.779, 123.68]
std = [58.393, 57.12, 57.375]
eigval = [55.46, 4.794, 1.148]
eigvec = [[-0.5836, -0.6948, 0.4203],
          [-0.5808, -0.0045, -0.8140],
          [-0.5675, 0.7192, 0.4009]]


187808
1468
9885


In [4]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model, models, Input, regularizers
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3, DenseNet121, VGG16



class VGG16Net(Model):
    def __init__(self):
        super(VGG16Net, self).__init__()
        self.base_model = VGG16(weights='imagenet', include_top=False, pooling='max')
        # self.base_model.trainable = False
        for i in range(len(self.base_model.layers) - 2):  # print(len(model.layers))=23
            self.base_model.layers[i].trainable = False
        self.d1 = Dense(units=hash_bit, activation=tf.nn.sigmoid)
        self.d2 = Dense(units=num_class, activation=tf.nn.softmax)

    def call(self, x):
        x = self.base_model(x)
        hash_code = self.d1(x)
        prediction = self.d2(hash_code)
        return prediction, hash_code

class VGG19Net(Model):
    def __init__(self):
        super(VGG19Net, self).__init__()
        self.base_model = VGG19(weights='imagenet', include_top=False, pooling='max')
        # self.base_model.trainable = False
        #for i in range(len(self.base_model.layers)):
        #    self.base_model.layers[i].trainable = False
        self.d1 = Dense(units=hash_bit, activation=None)
        self.bn1 = BatchNormalization()
        self.ac1 = tf.keras.layers.Activation("sigmoid")
        self.d2 = Dense(units=num_class, activation=tf.nn.softmax)

    def call(self, x):
        x = self.base_model(x)
        hash_code = self.d1(x)
        hash_code = self.bn1(hash_code)
        hash_code = self.ac1(hash_code)
        prediction = self.d2(hash_code)
        return prediction, hash_code    
    


class VGG19Net(Model):
    def __init__(self):
        super(VGG19Net, self).__init__()
        self.base_model = VGG19(weights='imagenet', include_top=False, pooling='max')
        # self.base_model.trainable = False
        #for i in range(len(self.base_model.layers)):
        #    self.base_model.layers[i].trainable = False
        self.d1 = Dense(units=hash_bit, activation=None)
        self.bn1 = BatchNormalization()
        self.ac1 = tf.keras.layers.Activation("sigmoid")
        self.d2 = Dense(units=num_class, activation=tf.nn.softmax)

    def call(self, x):
        x = self.base_model(x)
        hash_code = self.d1(x)
        hash_code = self.bn1(hash_code)
        hash_code = self.ac1(hash_code)
        prediction = self.d2(hash_code)
        return prediction, hash_code    



def Resnet_Model():
    base_model = ResNet50(weights='imagenet', include_top=False, layers=tf.keras.layers, input_shape=(224, 224, 3))
    print(len(base_model.layers))  # 175
    #for layer in base_model.layers[:170]:
    #    layer.trainable = False
    #for layer in base_model.layers[170:]:
    #    layer.trainable = True
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D(name='average_pool')(x)
    x = tf.keras.layers.Flatten(name='flatten')(x)
    hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None, kernel_regularizer=regularizers.l2(0.001))(x)
    # hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None)(x)
    hash_value = tf.keras.layers.BatchNormalization()(hash_value)
    hash_value = tf.keras.layers.Activation("sigmoid")(hash_value)
    prediction = tf.keras.layers.Dense(units=num_class, activation="softmax",
                                       kernel_regularizer=regularizers.l2(0.001))(hash_value)
    # prediction = tf.keras.layers.Dense(units=num_class, activation="softmax")(hash_value)
    model = models.Model(inputs=base_model.input, outputs=[prediction, hash_value])
    return model


def DenseNet_Model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    print(len(base_model.layers))  # 427
    for layer in base_model.layers[:415]:
       layer.trainable = False
    for layer in base_model.layers[415:]:
       layer.trainable = True
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D(name='average_pool')(x)
    x = tf.keras.layers.Flatten(name='flatten')(x)
    #x = tf.keras.layers.Dropout(0.5)(x)
    hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None)(x)
    hash_value = tf.keras.layers.BatchNormalization()(hash_value)
    hash_value = tf.keras.layers.Activation("sigmoid")(hash_value)
    prediction = tf.keras.layers.Dense(units=num_class, activation="softmax")(hash_value)
    model = models.Model(inputs=base_model.input, outputs=[prediction, hash_value])
    return model




In [5]:
# @title Default title text
#!/usr/bin/python3
#coding=utf-8
#coding=gbk
!pip install opencv-python
import tensorflow as tf
import numpy as np
import cv2


def load_list(list_path):
    images = []
    labels = []
    print(list_path)
    with open(list_path, 'r') as f:
        for line in f:
            lines = line.replace('\n', '').split('\t')
            if len(lines) == 1:
                lines = line.replace('\n', '').split()
            # print(lines)
            # images.append(os.path.join(image_root_path, line[0]))
            images.append(lines[0])
            labels.append(int(lines[1]))
    return images, labels


def random_size(image, target_size=None):
    height, width, _ = np.shape(image)
    if height < width:
        size_ratio = target_size / height
    else:
        size_ratio = target_size / width
    resize_shape = (int(width * size_ratio), int(height * size_ratio))
    return cv2.resize(image, resize_shape)


def normalize(image):
    # iamgenet归一化中用到的数�?
    mean = [103.939, 116.779, 123.68]
    std = [58.393, 57.12, 57.375]
    for i in range(3):
        image[..., i] = (image[..., i] - mean[i]) / std[i]
    return image


def random_crop(image):
    height, width, _ = np.shape(image)
    if height < width:
        num = width - height
        import random
        value = random.randint(0, num)
        value //= 2
        image = image[:, value:224 + value]
        image = cv2.resize(image, (224, 224))

    else:
        num = height - width
        import random
        value = random.randint(0, num)
        value //= 2
        image = image[value:224 + value, :]
        image = cv2.resize(image, (224, 224))
    return image

#�������ǵ�jpg�ļ����н��룬�����ά����
def load_preprosess_image(path,label, category_num):
    #��ȡ·��
    image = cv2.imread(path.numpy().decode()).astype(np.float32)
    #����
    #image=tf.image.decode_jpeg(image,channels=3)#��ɫͼ��Ϊ3��channel
    #��ͼ��ı�Ϊͬ���Ĵ�С�����òü�����Ť��,����Ӧ����Ť��
    image=tf.image.resize(image,[256,256])
    #����ü�ͼ��
    image=tf.image.random_crop(image,[224,224,3])
    #������·�תͼ��
    image=tf.image.random_flip_left_right(image)
    #������·�ת
    image=tf.image.random_flip_up_down(image)
    #����ı�ͼ�������
    image=tf.image.random_brightness(image,0.5)
    #����ı�Աȶ�
    image=tf.image.random_contrast(image,0,1)
    #�ı���������
    image=tf.cast(image,tf.float32)
    #��ͼ����й�һ��
    image=image/255
    #���ڻ���Ҫ��label���д��������������б�[1,2,3],
    #��Ҫ���[[1].[2].[3]]
    label_one_hot = np.zeros(category_num)
    label_one_hot[label] = 1.0
    return image, label_one_hot


def load_image(image_path, label, category_num):
    #print(image_path)
    #image_path = "/home/ubuntu/shy/tf2_learn/21_pokemon/" + image_path
    #print(image_path)
    image = cv2.imread(image_path.numpy().decode()).astype(np.float32)
    #print(image.shape)
    image = random_size(image, target_size=256)
    image = random_crop(image)
    image = normalize(image)
    label_one_hot = np.zeros(category_num)
    label_one_hot[label] = 1.0

    return image, label_one_hot


def train_iterator(train_list_path, class_nums, batch_size):
    # train_data_path路径TXT文件
    # images, labels = load_list(train_data_path) # jfkdsjf.jpg 2
    images, labels = load_list(train_list_path)  # jfkdsjf.jpg 2
    print(images)
    #img = []
    #for image in images:
    #  image = "/home/ubuntu/shy/tf2_learn/21_pokemon/" + image
     # img.append(image)
    # print(images)
    print(len(images), len(labels))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, class_nums], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #dataset = dataset.shuffle(len(images))  # 测试集不需�?
    dataset = dataset.repeat()  # 测试集不需�?
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    #return it, images
    return it

def test_iterator(test_list_path, class_nums, batch_size):
    images, labels = load_list(test_list_path)  # jfkdsjf.jpg 2
    print(len(images))
    print(len(labels))
    #print(images, labels)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, class_nums], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    return it, images
    #print(images)
    return it


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 1.9 MB/s eta 0:00:0000:0100:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(rendera

In [6]:
#!/usr/bin/python?
# -*- coding: utf-8 -*-import tensorflow as tf
#load load_file config_file

import numpy as np
import pickle as p
from tqdm import tqdm
import os
import cv2
from tensorflow.keras import models, optimizers, Sequential
from tensorflow.keras.layers import Input

import tensorflow as tf
import math


def getBinaryTensor(imgTensor, boundary=0.5):
    one = tf.ones_like(imgTensor)
    zero = tf.zeros_like(imgTensor)
    return tf.where(imgTensor > boundary, one, zero)


def hash_loss_fn(hash_input):
    loss1 = -1 * tf.reduce_mean(tf.square(hash_input - 0.5)) + 0.25  # ���ֵΪ0.25
    loss2 = tf.reduce_mean(tf.square(tf.reduce_mean(hash_input, axis=1) - 0.5))
    return loss1 + loss2


def cross_entropy(y_true, y_pred):
    cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(cross_entropy)


def l2_loss(model, weights=weight_decay):
    variable_list = []
    for v in model.trainable_variables:
        if 'kernel' in v.name:
            variable_list.append(tf.nn.l2_loss(v))
    return tf.add_n(variable_list) * weights


def accuracy(y_true, y_pred):
    correct_num = tf.equal(tf.argmax(y_true, -1), tf.argmax(y_pred, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_num, dtype=tf.float32))
    return accuracy


@tf.function
def test_step(model, x, y):
    prediction, hash_value = model(x, training=False)  # �޸�
    ce = cross_entropy(y, prediction)
    return ce, prediction, hash_value


def test(model, test_iter,xyz):
    sum_loss = 0
    sum_accuracy = 0
    count = 0
    hashh_train=np.array([])
    labell_train=np.array([])
    test_data_iterator, images = test_iterator(xyz, num_class, batch_size)
    filewriter = open('hash_code_test_resnet.txt', 'w')
    for i in tqdm(range(test_iter)):
        x, y = test_data_iterator.next()
        x = tf.cast(x, tf.float32)  # �����Լ��е�ͼ������float32
        loss, prediction, hash_value = test_step(model, x, y)
        sum_loss += loss
        sum_accuracy += accuracy(y, prediction)
        hash_value = getBinaryTensor(hash_value)
        hashh_train=np.append(hashh_train,hash_value)
        labell_train=np.append(labell_train,y)

        for j in range(hash_value.shape[0]):
            filewriter.writelines(  str(hash_value[j].numpy())  + "\n")
            count += 1
    print('test, loss:%f, accuracy:%f' %
          (sum_loss / test_iter, sum_accuracy / test_iter))
    return hashh_train,labell_train


if __name__ == '__main__':
    import os

    # get model
    #img_input = Input(shape=(224, 224, 3))
    #net = VGG19Net()
    #net.build((None,224,224,3))
    #output = net(img_input)
    #model = models.Model(img_input, output)
    # os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    # physical_devices = tf.config.experimental.list_physical_devices(device_type='GPU')
    # tf.config.experimental.set_memory_growth(device=physical_devices[0], enable=True)

    model = Resnet_Model()
    model.build((None, 224, 224, 3))

    checkpoint_save_path = "b200834cs/resnet/"
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    print('-------------load the model-----------------')
    if checkpoint.restore(tf.train.latest_checkpoint(checkpoint_save_path)):
       print("succesful")
    print("loaded the model " + checkpoint_save_path + "model.ckpt-4")
    train_acc,traint = test(model, iterations_per_epoch,"Downloads/train.txt")
    print(train_acc)
    print(traint)
    traint_binary_resnet=train_acc.reshape(int(len(train_acc)/512),512)
    test_acc,testt = test(model, test_iterations,"Downloads/test.txt")
    print(test_acc)
    print(testt)
    testt_binary_resnet=test_acc.reshape(int(len(test_acc)/512),512)
    
    

2024-05-05 17:33:03.604246: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-05-05 17:33:03.647623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:4b:00.0
2024-05-05 17:33:03.647684: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-05-05 17:33:03.729824: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-05-05 17:33:03.798486: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024-05-05 17:33:03.816270: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2024-05-05 17:33:03.831255: I tensorflow/stream_executor/platform/de

94773248/94765736 [==============================] - 9s 0us/step
175
-------------load the model-----------------
succesful
loaded the model b200834cs/resnet/model.ckpt-4
Downloads/train.txt
187808
187808


  0%|                                                                                                | 0/1468 [00:00<?, ?it/s]2024-05-05 17:33:35.676688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-05-05 17:33:39.621159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
100%|█████████████████████████████████████████████████████████████████████████████████████| 1468/1468 [35:55<00:00,  1.47s/it]


test, loss:0.216974, accuracy:0.949804
[1. 1. 0. ... 1. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
Downloads/test.txt
9885
9885


100%|█████████████████████████████████████████████████████████████████████████████████████████| 78/78 [01:44<00:00,  1.34s/it]

test, loss:0.369534, accuracy:0.909272
[0. 0. 0. ... 0. 0. 1.]
[0. 0. 0. ... 0. 0. 0.]


In [7]:
#!/usr/bin/python?
# -*- coding: utf-8 -*-import tensorflow as tf
#load load_file config_file

import numpy as np
import pickle as p
from tqdm import tqdm
import os
import cv2
from tensorflow.keras import models, optimizers, Sequential
from tensorflow.keras.layers import Input

import tensorflow as tf
import math


def getBinaryTensor(imgTensor, boundary=0.5):
    one = tf.ones_like(imgTensor)
    zero = tf.zeros_like(imgTensor)
    return tf.where(imgTensor > boundary, one, zero)


def hash_loss_fn(hash_input):
    loss1 = -1 * tf.reduce_mean(tf.square(hash_input - 0.5)) + 0.25  # ���ֵΪ0.25
    loss2 = tf.reduce_mean(tf.square(tf.reduce_mean(hash_input, axis=1) - 0.5))
    return loss1 + loss2


def cross_entropy(y_true, y_pred):
    cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(cross_entropy)


def l2_loss(model, weights=weight_decay):
    variable_list = []
    for v in model.trainable_variables:
        if 'kernel' in v.name:
            variable_list.append(tf.nn.l2_loss(v))
    return tf.add_n(variable_list) * weights


def accuracy(y_true, y_pred):
    correct_num = tf.equal(tf.argmax(y_true, -1), tf.argmax(y_pred, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_num, dtype=tf.float32))
    return accuracy


@tf.function
def test_step(model, x, y):
    prediction, hash_value = model(x, training=False)  # �޸�
    ce = cross_entropy(y, prediction)
    return ce, prediction, hash_value


def test(model, test_iter,xyz):
    sum_loss = 0
    sum_accuracy = 0
    count = 0
    hashh_train=np.array([])
    labell_train=np.array([])
    test_data_iterator, images = test_iterator(xyz, num_class, batch_size)
    filewriter = open('hash_code_test_resnet.txt', 'w')
    for i in tqdm(range(test_iter)):
        x, y = test_data_iterator.next()
        x = tf.cast(x, tf.float32)  # �����Լ��е�ͼ������float32
        loss, prediction, hash_value = test_step(model, x, y)
        sum_loss += loss
        sum_accuracy += accuracy(y, prediction)
        hash_value = getBinaryTensor(hash_value)
        hashh_train=np.append(hashh_train,hash_value)
        labell_train=np.append(labell_train,y)

        for j in range(hash_value.shape[0]):
            filewriter.writelines(  str(hash_value[j].numpy())  + "\n")
            count += 1
    print('test, loss:%f, accuracy:%f' %
          (sum_loss / test_iter, sum_accuracy / test_iter))
    return hashh_train,labell_train


if __name__ == '__main__':
    import os

    # get model
    #img_input = Input(shape=(224, 224, 3))
    #net = VGG19Net()
    #net.build((None,224,224,3))
    #output = net(img_input)
    #model = models.Model(img_input, output)
#     os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#     physical_devices = tf.config.experimental.list_physical_devices(device_type='GPU')
#     tf.config.experimental.set_memory_growth(device=physical_devices[0], enable=True)

    model = DenseNet_Model()
    model.build((None, 224, 224, 3))

    checkpoint_save_path = "b200834cs/densenet/"
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    print('-------------load the model-----------------')
    if checkpoint.restore(tf.train.latest_checkpoint(checkpoint_save_path)):
       print("succesful")
    print("loaded the model " + checkpoint_save_path + "model.ckpt-4")
    train_acc,traint = test(model, iterations_per_epoch,"Downloads/train.txt")
    print(train_acc)
    print(traint)
    traint_binary_densenet=train_acc.reshape(int(len(train_acc)/512),512)
    test_acc,testt = test(model, test_iterations-1,"Downloads/test.txt")
    print(test_acc)
    print(testt)
    testt_binary_densenet=test_acc.reshape(int(len(test_acc)/512),512)
    
    

29089792/29084464 [==============================] - 3s 0us/step
427
-------------load the model-----------------
succesful
loaded the model b200834cs/densenet/model.ckpt-4
Downloads/train.txt
187808
187808


100%|█████████████████████████████████████████████████████████████████████████████████████| 1468/1468 [36:36<00:00,  1.50s/it]


test, loss:3.143089, accuracy:0.302697
[0. 0. 1. ... 0. 0. 1.]
[0. 0. 0. ... 0. 0. 0.]
Downloads/test.txt
9885
9885


100%|█████████████████████████████████████████████████████████████████████████████████████████| 77/77 [01:44<00:00,  1.36s/it]

test, loss:3.226787, accuracy:0.283787
[1. 0. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]


In [8]:
#!/usr/bin/python?
# -*- coding: utf-8 -*-import tensorflow as tf
#load load_file config_file

import numpy as np
import pickle as p
from tqdm import tqdm
import os
import cv2
from tensorflow.keras import models, optimizers, Sequential
from tensorflow.keras.layers import Input

import tensorflow as tf
import math


def getBinaryTensor(imgTensor, boundary=0.5):
    one = tf.ones_like(imgTensor)
    zero = tf.zeros_like(imgTensor)
    return tf.where(imgTensor > boundary, one, zero)


def hash_loss_fn(hash_input):
    loss1 = -1 * tf.reduce_mean(tf.square(hash_input - 0.5)) + 0.25  # ���ֵΪ0.25
    loss2 = tf.reduce_mean(tf.square(tf.reduce_mean(hash_input, axis=1) - 0.5))
    return loss1 + loss2


def cross_entropy(y_true, y_pred):
    cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(cross_entropy)


def l2_loss(model, weights=weight_decay):
    variable_list = []
    for v in model.trainable_variables:
        if 'kernel' in v.name:
            variable_list.append(tf.nn.l2_loss(v))
    return tf.add_n(variable_list) * weights


def accuracy(y_true, y_pred):
    correct_num = tf.equal(tf.argmax(y_true, -1), tf.argmax(y_pred, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_num, dtype=tf.float32))
    return accuracy


@tf.function
def test_step(model, x, y):
    prediction, hash_value = model(x, training=False)  # �޸�
    ce = cross_entropy(y, prediction)
    return ce, prediction, hash_value


def test(model, test_iter,xyz):
    sum_loss = 0
    sum_accuracy = 0
    count = 0
    hashh_train=np.array([])
    labell_train=np.array([])
    test_data_iterator, images = test_iterator(xyz, num_class, batch_size)
    filewriter = open('hash_code_test_vgg19.txt', 'w')
    for i in tqdm(range(test_iter)):
        x, y = test_data_iterator.next()
        x = tf.cast(x, tf.float32)  # �����Լ��е�ͼ������float32
        loss, prediction, hash_value = test_step(model, x, y)
        sum_loss += loss
        sum_accuracy += accuracy(y, prediction)
        hash_value = getBinaryTensor(hash_value)
        hashh_train=np.append(hashh_train,hash_value)
        labell_train=np.append(labell_train,y)

        for j in range(hash_value.shape[0]):
            filewriter.writelines(  str(hash_value[j].numpy())  + "\n")
            count += 1
    print('test, loss:%f, accuracy:%f' %
          (sum_loss / test_iter, sum_accuracy / test_iter))
    return hashh_train,labell_train


if __name__ == '__main__':
    import os

    # get model
    #img_input = Input(shape=(224, 224, 3))
    #net = VGG19Net()
    #net.build((None,224,224,3))
    #output = net(img_input)
    #model = models.Model(img_input, output)
#     os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#     physical_devices = tf.config.experimental.list_physical_devices(device_type='GPU')
#     tf.config.experimental.set_memory_growth(device=physical_devices[0], enable=True)

    model = VGG19Net()
    model.build((None, 224, 224, 3))

    checkpoint_save_path = "b200834cs/vgg/"
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    print('-------------load the model-----------------')
    if checkpoint.restore(tf.train.latest_checkpoint(checkpoint_save_path)):
       print("succesful")
    print("loaded the model " + checkpoint_save_path + "model.ckpt-4")
    train_acc,traint = test(model, iterations_per_epoch,"Downloads/train.txt")
    print(train_acc)
    print(traint)
    traint_binary_vgg=train_acc.reshape(int(len(train_acc)/512),512)
    test_acc,testt = test(model, test_iterations,"Downloads/test.txt")
    print(test_acc)
    print(testt)
    testt_binary_vgg=test_acc.reshape(int(len(test_acc)/512),512)
    
    

80142336/80134624 [==============================] - 8s 0us/step
-------------load the model-----------------
succesful
loaded the model b200834cs/vgg/model.ckpt-4
Downloads/train.txt
187808
187808


100%|█████████████████████████████████████████████████████████████████████████████████████| 1468/1468 [36:46<00:00,  1.50s/it]


test, loss:0.293885, accuracy:0.934717
[0. 1. 0. ... 1. 1. 0.]
[0. 0. 0. ... 0. 0. 0.]
Downloads/test.txt
9885
9885


100%|█████████████████████████████████████████████████████████████████████████████████████████| 78/78 [01:47<00:00,  1.38s/it]

test, loss:0.447672, accuracy:0.896227
[0. 1. 1. ... 1. 0. 1.]
[0. 0. 0. ... 0. 0. 0.]


In [6]:
xx,traint_resnet=load_list("Downloads/train.txt")
yy,testt_resnet=load_list("Downloads/test.txt")
xx,traint_densenet=load_list("Downloads/train.txt")
yy,testt_densenet=load_list("Downloads/test.txt")
xx,traint_vgg=load_list("Downloads/train.txt")
yy,testt_vgg=load_list("Downloads/test.txt")
print(testt_resnet[:500])
print(len(traint_resnet))
print(len(testt_resnet))

Downloads/train.txt
Downloads/test.txt
Downloads/train.txt
Downloads/test.txt
Downloads/train.txt
Downloads/test.txt
[355, 177, 86, 177, 56, 445, 445, 263, 56, 56, 0, 355, 86, 56, 0, 177, 264, 264, 264, 0, 86, 0, 86, 355, 264, 264, 178, 56, 446, 86, 0, 178, 178, 446, 56, 0, 86, 355, 86, 178, 56, 86, 86, 446, 264, 446, 355, 86, 264, 264, 0, 178, 57, 57, 86, 0, 264, 264, 57, 178, 178, 264, 264, 57, 87, 264, 446, 178, 57, 178, 355, 87, 87, 1, 264, 57, 57, 178, 87, 57, 1, 178, 57, 57, 57, 446, 1, 87, 446, 87, 57, 57, 446, 446, 178, 355, 87, 178, 355, 264, 1, 446, 446, 87, 57, 264, 57, 57, 178, 1, 264, 265, 355, 1, 265, 355, 87, 355, 87, 1, 1, 265, 1, 1, 446, 265, 355, 87, 87, 355, 446, 178, 355, 1, 178, 178, 356, 356, 178, 356, 356, 1, 446, 57, 1, 356, 265, 57, 356, 57, 446, 356, 265, 265, 178, 265, 265, 446, 87, 356, 87, 1, 87, 87, 1, 87, 178, 265, 356, 356, 446, 356, 446, 1, 446, 58, 178, 356, 356, 178, 58, 58, 265, 1, 178, 265, 356, 178, 356, 447, 58, 1, 447, 87, 265, 447, 58, 1, 178, 4

In [14]:
import numpy as np

# Save array
np.save('Downloads/vgg_train.npy', traint_binary_vgg)
np.save('Downloads/vgg_test.npy', testt_binary_vgg)
np.save('Downloads/resnet_train.npy', traint_binary_resnet)
np.save('Downloads/resnet_test.npy', testt_binary_resnet)
np.save('Downloads/densenet_train.npy', traint_binary_densenet)
np.save('Downloads/densenet_test.npy', testt_binary_densenet)

# Load array



In [7]:
traintt_binary_vgg = np.load('Downloads/vgg_train.npy')
testtt_binary_vgg = np.load('Downloads/vgg_test.npy')
traintt_binary_resnet = np.load('Downloads/resnet_train.npy')
testtt_binary_resnet = np.load('Downloads/resnet_test.npy')
traintt_binary_densenet = np.load('Downloads/densenet_train.npy')
testtt_binary_densenet = np.load('Downloads/densenet_test.npy')


In [8]:
print(len(traintt_binary_vgg))
print(len(testtt_binary_densenet))
print(len(traintt_binary_resnet))

187808
9856
187808


In [9]:
import numpy as np
from tqdm import tqdm
import time

np.set_printoptions(threshold=np.inf) ##no of elements in an array should be displayed when we print an array
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##calculate hamming distance between two arrays
def hanming(v1, v2):
    tt = 1.0 * np.sum(v1 != v2, axis=1)
    return tt

##calculating the element wise divison
def true_divide(buffer_yes, Ns):
    P = np.true_divide(np.cumsum(buffer_yes), np.array(Ns))
    return P


## return the indices of the elements in ascending order
def argsort(similarity):
    return np.argsort(similarity, kind='mergesort')


def sum(arr):
    res = 0
    for i in range(len(arr)):
        res += arr[i]
    return res


def hanming_normalazition(arr):
    res = []
    # print("汉明归一化循环次数，",len(arr))
    for i in range(len(arr)):
        res.append(round(1 - (arr[i] / 48), 4))
    return res
    # return [round(1 - (i / 48), 4) for i in arr]


def cumsum(arr):
    return np.cumsum(arr)


def zeros(K):
    return np.zeros(K)

## train_label[0], y2_1, K, test_label[0][i], buffer_yes_1
def fun(label, y, K, query_label, buffer_yes):
    # print("fun函数  循环次数：", K)
    for j in range(0, K):
        # print(j)
        # 检索出来的标签
        retrieval_label = label[y[j]]
        if query_label == retrieval_label:
            buffer_yes[j] = 1
    return buffer_yes


def produce_hanming_list(arr):
    temp = []
    for i in range(len(arr)):
        temp.append(arr[i])
    return temp


def hanming_matrix(hanming_normalazition):
    hanming = []
    for i in hanming_normalazition:
        hanming.append(hanming_normalazition[i])
    return hanming

## contains three image paths containing k images each and it corresponding hamming distances and it will return the k images among which are least hamming distance
def list_add(K, pic_path1, pic_path2, pic_path3, hanming_matrix1, hanming_matrix2, hanming_matrix3):
    list1_1, list1_2, list1_3, = [], [], []
    # 三个for循环比较慢
    # 键 1-n   值:正确图片的索引
    # print("pic_path",pic_path1)
    # print(pic_path1.values())
    for k, v in pic_path1.items():
        # print("pic_path1.index(p1)",pic_path1.get(v))
        # print("v",v)
        # print(pic_path1[k])
        result1 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_1.append(result1)
    # print(list1_1)
    for k, v in pic_path2.items():
        result2 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_2.append(result2)
    for k, v in pic_path3.items():
        result3 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_3.append(result3)
    path = list(pic_path1.keys())[:K] + list(pic_path2.keys())[:K] + list(pic_path3.keys())[:K]
    new_list = list1_1 + list1_2 + list1_3
    return path, new_list

##train_label[0], train_base_dic, final_path_dic, final_index_dict

def fn(train_label, train_base_dic, final_path_dic, final_index_dict):
    # print("final_path", final_path)
    # base_x = train_base[0].tolist()
    # base_x = [int(i) for i in base_x]   # base_x   0-49999
    temp_x = []
    for _, v in final_index_dict.items():
        # index_x = final_path_dic.get(v)
        temp_x.append(train_label[train_base_dic.get(final_path_dic.get(v))])

    return temp_x[::-1]


##DataLabel, DataBase, DataPath, TestLabel, TestData, TestPath, 1000754
def precision(train_label, train_binary, train_base, test_label, test_binary, test_query, top_k):
    """
    train_label:训练集图片的标签
    train_binary:训练集图片的二值码
    test_label:测试集图片的标签
    test_binary:测试集图片的二值码
    top_k:查询返回图片（top_k）数量的map值
    """
    K = top_k
    QueryTimes = test_binary[0].shape[0] - 128 
    print(QueryTimes)# 获得行数 //size of the array of the first index
    AP_1 = np.zeros((QueryTimes, 1)) # AP_1 is an array containing  zeros same as number of testing data images
    AP_2 = np.zeros((QueryTimes, 1))
    AP_3 = np.zeros((QueryTimes, 1))

    # [1,2,....K]
    Ns = [i for i in range(1, K + 1)]
    train_base = [int(i) for i in train_base[0]]
    # print("train_base",train_base)
    train_base = [int(i) for i in train_base]
    train_base_dic = {} #dictionary with key value pairs
    for i in range(len(train_base)):
        train_base_dic[i] = i
    # print("train_base",train_base_dic)
    ap1 = 0
    for i in tqdm(range(0, QueryTimes)):
        # for i in range(0, QueryTimes):
        # 查询集   真实标签
        query_label_3 = test_label[2][i]

#         print("query_label_3", query_label_3)
        # print(test_query[i])

        # 真实标签的哈希码
        query_binary_1 = test_binary[0][i]  # 去一个测试二值码
        query_binary_2 = test_binary[1][i]
        query_binary_3 = test_binary[2][i]

        # (6144,)  汉明矩阵  乱序

        similarity_1 = hanming(train_binary[0], query_binary_1)
        similarity_2 = hanming(train_binary[1], query_binary_2)
        similarity_3 = hanming(train_binary[2], query_binary_3)

        # 汉明矩阵排序，返回汉明距离从小到大的索引  即正确的索引

        y2_1 = argsort(similarity_1)
        y2_2 = argsort(similarity_2)
        y2_3 = argsort(similarity_3)# y2 contains the index values in ascending order of the similarity values of the array

        # (6144,)
        # print(y2_1[:10])

        buffer_yes_1 = zeros(K)
        buffer_yes_2 = zeros(K)
        buffer_yes_3 = zeros(K)

        buffer_yes_1 = fun(train_label[0], y2_1, K, test_label[0][i], buffer_yes_1)  ## finds whether the test label matches with label of top k images of the trained_dataset according to hamming distance or not
        buffer_yes_2 = fun(train_label[1], y2_2, K, test_label[1][i], buffer_yes_2) ## is an array of size k contains value 0- mismatch 1- match
        buffer_yes_3 = fun(train_label[2], y2_3, K, test_label[2][i], buffer_yes_3)

        P_1 = true_divide(buffer_yes_1, Ns)
        P_2 = true_divide(buffer_yes_2, Ns)
        P_3 = true_divide(buffer_yes_3, Ns)

        if sum(buffer_yes_1) == 0:
            AP_1[i] = 0
        else:
            AP_1[i] = sum(np.array(P_1) * np.array(buffer_yes_1)) / sum(buffer_yes_1)

        if sum(buffer_yes_2) == 0:
            AP_2[i] = 0
        else:
            AP_2[i] = sum(np.array(P_2) * np.array(buffer_yes_2)) / sum(buffer_yes_2)

        if sum(buffer_yes_3) == 0:
            AP_3[i] = 0
        else:
            AP_3[i] = sum(np.array(P_3) * np.array(buffer_yes_3)) / sum(buffer_yes_3)

        # 排序后的汉明距离矩阵

        order_map_1 = [similarity_1[i] for i in y2_1] ## contain values of hamming distance in ascending order
        # print("order_map_1", order_map_1)
        order_map_2 = [similarity_2[i] for i in y2_2]
        order_map_3 = [similarity_3[i] for i in y2_3]

        pic_path1 = {}
        pic_path2 = {}
        pic_path3 = {}

        for k, v in enumerate(y2_1, 0):
            # print("k",k)
            # print("v",v)
            pic_path1[v] = k  ## value- position of the similarity value  key-- index value

        # for step,k in enumerate(y2_1):
        #     print("step",step)
        #     print("k",k)
        #     pic_path1[k] = step
        for k, v in enumerate(y2_2, 0):
            pic_path2[v] = k
        for k, v in enumerate(y2_3, 0):
            pic_path3[v] = k

        hanming_normalazition1 = hanming_normalazition(order_map_1)

        hanming_normalazition2 = hanming_normalazition(order_map_2)
        hanming_normalazition3 = hanming_normalazition(order_map_3)

        # print("hanming_normalazition1", hanming_normalazition1)

        # 汉明距离归一化 * 精度
        hanming_matrix1 = [i * acc[0] for i in hanming_normalazition1]
        hanming_matrix2 = [i * acc[1] for i in hanming_normalazition2]
        hanming_matrix3 = [i * acc[2] for i in hanming_normalazition3]

        path, new_list = list_add(K, pic_path1, pic_path2, pic_path3, hanming_matrix1, hanming_matrix2, hanming_matrix3)

        t1 = time.time()

        dictionary = {}
        for step, p in enumerate(path):
            value = dictionary.get(p, [])
            value.append(step)
            dictionary[p] = value
        # print("dictionary",dictionary)
        path_arr = [None for _ in range(len(path))]
        num_arr = [0 for _ in range(len(path))]

        for k, v in dictionary.items():
            path_arr[v[0]] = k
            num_arr[v[0]] = new_list[v[0]]

        final_path = [i for i in path_arr if i is not None][:K]
        final_path_dic = {}
        for step, r in enumerate(final_path):
            final_path_dic[step] = r
        # print("final_path_dic",final_path_dic)
        final_num = [i for i in num_arr if i][:K]

        final_index = argsort(final_num)
        final_index_dict = {}
        for step, r in enumerate(final_index):
            final_index_dict[step] = r

        lable_final = fn(train_label[0], train_base_dic, final_path_dic, final_index_dict)

#         print("lable_final", lable_final)
        count = 0.0
        sum_value = 0.0
        for step, r in enumerate(lable_final[:top_k]):
            step += 1
#             print(r)
            if str(query_label_3) == str(r):
                count += 1
                sum_value += count / step
        if count :
#             print('@@@@@')
            ap = sum_value / count
        else:
            ap = 0
        ap1 += ap
#         print("ap", ap)
#         print("AP_1[i]", AP_1[i])
#         print("AP_2[i]", AP_2[i])
#         print("AP_3[i]", AP_3[i])
        t2 = time.time()
        #print("AP时间", t2 - t1)

    map_1 = np.mean(AP_1)
    map_2 = np.mean(AP_2)
    map_3 = np.mean(AP_3)
    ap1 = ap1 / QueryTimes

    # return precision_at_k, map
    return map_1, map_2, map_3, ap1


if __name__ == '__main__':
    # vgg resnet dense
    acc = [0.669, 0.692, 0.705]
    # 测试集
    # F:\project\produce_hash\NPY
    # F:\project\produce_hash\MAP_集成.py
#     traint_binary_densenet=traint_binary_resnet
#     traint_densenet=traint_resnet
#     testt_binary_densenet=testt_binary_resnet
#     testt_densenet=testt_resnet
    
    
    TestData_VGG = traintt_binary_vgg
    TestLabel_VGG = traint_vgg
    TestPath_VGG = traint_vgg
    # 训练集
    DataBase_VGG = testtt_binary_vgg[:1024]
    DataLabel_VGG = testt_vgg[:1024]
    DataPath_VGG = testt_vgg[:1024]

    # 测试集
    TestData_Resnet = traintt_binary_resnet
    TestLabel_Resnet = traint_resnet
    TestPath_Resnet = traint_resnet
    # 训练集
    DataBase_Resnet = testtt_binary_resnet[:1024]
    DataLabel_Resnet = testt_resnet[:1024]
    DataPath_Resnet = testt_resnet[:1024]

    # 测试集
    TestData_Desnet = traintt_binary_densenet
    TestLabel_Desnet = traint_densenet
    TestPath_Desnet = traint_densenet
    # 训练集
    DataBase_Desnet = testtt_binary_densenet[:1024]
    DataLabel_Desnet = testt_densenet[:1024]
    DataPath_Desnet = testt_densenet[:1024]

    DataLabel = [DataLabel_VGG, DataLabel_Resnet, DataLabel_Desnet]
    DataBase = [DataBase_VGG, DataBase_Resnet, DataBase_Desnet]
    DataPath = [DataPath_VGG, DataPath_Resnet, DataPath_Desnet]

    TestLabel = [TestLabel_VGG, TestLabel_Resnet, TestLabel_Desnet]
    TestData = [TestData_VGG, TestData_Resnet, TestData_Desnet]
    TestPath = [TestPath_VGG, TestPath_Resnet, TestPath_Desnet]
    print(len(TestLabel[0]))
    map = precision( TestLabel, TestData, TestPath,DataLabel, DataBase, DataPath, 30)

    filewrite = open("map_sun397_48bit_top1000754_bit.txt", "w")
    for i in map:
        filewrite.write("map: " + str(i) + "\n")
        filewrite.write("bit_num: " + "48\n")
        print(i)


187808
896


  0%|                                                                                                 | 0/896 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import numpy as np
from tqdm import tqdm
import time

np.set_printoptions(threshold=np.inf) ##no of elements in an array should be displayed when we print an array
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##calculate hamming distance between two arrays
def hanming(v1, v2):
    tt = 1.0 * np.sum(v1 != v2, axis=1)
    return tt

##calculating the element wise divison
def true_divide(buffer_yes, Ns):
    P = np.true_divide(np.cumsum(buffer_yes), np.array(Ns))
    return P


## return the indices of the elements in ascending order
def argsort(similarity):
    return np.argsort(similarity, kind='mergesort')


def sum(arr):
    res = 0
    for i in range(len(arr)):
        res += arr[i]
    return res


def hanming_normalazition(arr):
    res = []
    # print("汉明归一化循环次数，",len(arr))
    for i in range(len(arr)):
        res.append(round(1 - (arr[i] / 48), 4))
    return res
    # return [round(1 - (i / 48), 4) for i in arr]


def cumsum(arr):
    return np.cumsum(arr)


def zeros(K):
    return np.zeros(K)

## train_label[0], y2_1, K, test_label[0][i], buffer_yes_1
def fun(label, y, K, query_label, buffer_yes):
    # print("fun函数  循环次数：", K)
    for j in range(0, K):
        # print(j)
        # 检索出来的标签
        retrieval_label = label[y[j]]
        if query_label == retrieval_label:
            buffer_yes[j] = 1
    return buffer_yes


def produce_hanming_list(arr):
    temp = []
    for i in range(len(arr)):
        temp.append(arr[i])
    return temp


def hanming_matrix(hanming_normalazition):
    hanming = []
    for i in hanming_normalazition:
        hanming.append(hanming_normalazition[i])
    return hanming

## contains three image paths containing k images each and it corresponding hamming distances and it will return the k images among which are least hamming distance
def list_add(K, pic_path1, pic_path2, pic_path3, hanming_matrix1, hanming_matrix2, hanming_matrix3):
    list1_1, list1_2, list1_3, = [], [], []
    # 三个for循环比较慢
    # 键 1-n   值:正确图片的索引
    # print("pic_path",pic_path1)
    # print(pic_path1.values())
    for k, v in pic_path1.items():
        # print("pic_path1.index(p1)",pic_path1.get(v))
        # print("v",v)
        # print(pic_path1[k])
        result1 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_1.append(result1)
    # print(list1_1)
    for k, v in pic_path2.items():
        result2 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_2.append(result2)
    for k, v in pic_path3.items():
        result3 = (hanming_matrix1[pic_path1[k]] + hanming_matrix2[pic_path2[k]] + hanming_matrix3[
            pic_path3[k]]) / 3
        list1_3.append(result3)
    path = list(pic_path1.keys())[:K] + list(pic_path2.keys())[:K] + list(pic_path3.keys())[:K]
    new_list = list1_1 + list1_2 + list1_3
    return path, new_list

##train_label[0], train_base_dic, final_path_dic, final_index_dict

def fn(train_label, train_base_dic, final_path_dic, final_index_dict):
    # print("final_path", final_path)
    # base_x = train_base[0].tolist()
    # base_x = [int(i) for i in base_x]   # base_x   0-49999
    temp_x = []
    for _, v in final_index_dict.items():
        # index_x = final_path_dic.get(v)
        temp_x.append(train_label[train_base_dic.get(final_path_dic.get(v))])

    return temp_x[::-1]


##DataLabel, DataBase, DataPath, TestLabel, TestData, TestPath, 1000754
def precision(train_label, train_binary, train_base, test_label, test_binary, test_query, top_k):
    """
    train_label:训练集图片的标签
    train_binary:训练集图片的二值码
    test_label:测试集图片的标签
    test_binary:测试集图片的二值码
    top_k:查询返回图片（top_k）数量的map值
    """
    K = top_k
    QueryTimes = test_binary[0].shape[0] - 128 
    print(QueryTimes)# 获得行数 //size of the array of the first index
    AP_1 = np.zeros((QueryTimes, 1)) # AP_1 is an array containing  zeros same as number of testing data images
    AP_2 = np.zeros((QueryTimes, 1))
    AP_3 = np.zeros((QueryTimes, 1))
    recal_1= np.zeros((QueryTimes, 1))
    recal_2= np.zeros((QueryTimes, 1)) 
    recal_3= np.zeros((QueryTimes, 1))
    f1_1= np.zeros((QueryTimes, 1))
    f1_2= np.zeros((QueryTimes, 1))
    f1_3= np.zeros((QueryTimes, 1))
    # [1,2,....K]
    Ns = [i for i in range(1, K + 1)]
    train_base = [int(i) for i in train_base[0]]
    # print("train_base",train_base)
    train_base = [int(i) for i in train_base]
    train_base_dic = {} #dictionary with key value pairs
    for i in range(len(train_base)):
        train_base_dic[i] = i
    # print("train_base",train_base_dic)
    ap1 = 0
    recal = 0
    F1score = 0
#     F1score
    for i in tqdm(range(0, QueryTimes)):
        # for i in range(0, QueryTimes):
        # 查询集   真实标签
        query_label_3 = test_label[2][i]

#         print("query_label_3", query_label_3)
        # print(test_query[i])

        # 真实标签的哈希码
        query_binary_1 = test_binary[0][i]  # 去一个测试二值码
        query_binary_2 = test_binary[1][i]
        query_binary_3 = test_binary[2][i]

        # (6144,)  汉明矩阵  乱序

        similarity_1 = hanming(train_binary[0], query_binary_1)
        similarity_2 = hanming(train_binary[1], query_binary_2)
        similarity_3 = hanming(train_binary[2], query_binary_3)

        # 汉明矩阵排序，返回汉明距离从小到大的索引  即正确的索引

        y2_1 = argsort(similarity_1)
        y2_2 = argsort(similarity_2)
        y2_3 = argsort(similarity_3)# y2 contains the index values in ascending order of the similarity values of the array

        # (6144,)
        # print(y2_1[:10])

        buffer_yes_1 = zeros(K)
        buffer_yes_2 = zeros(K)
        buffer_yes_3 = zeros(K)

        buffer_yes_1 = fun(train_label[0], y2_1, K, test_label[0][i], buffer_yes_1)  ## finds whether the test label matches with label of top k images of the trained_dataset according to hamming distance or not
        buffer_yes_2 = fun(train_label[1], y2_2, K, test_label[1][i], buffer_yes_2) ## is an array of size k contains value 0- mismatch 1- match
        buffer_yes_3 = fun(train_label[2], y2_3, K, test_label[2][i], buffer_yes_3)

        P_1 = true_divide(buffer_yes_1, Ns)
        P_2 = true_divide(buffer_yes_2, Ns)
        P_3 = true_divide(buffer_yes_3, Ns)

        if sum(buffer_yes_1) == 0:
            AP_1[i] = 0
            recal_1[i]=0
            f1_1[i]=0
        else:
            AP_1[i] = sum(np.array(P_1) * np.array(buffer_yes_1)) / sum(buffer_yes_1)
            recal_1[i]=AP_1[i]*sum(buffer_yes_1)/400
            f1_1[i]=2*AP_1[i]*recal_1[i]/(AP_1[i]+recal_1[i])

        if sum(buffer_yes_2) == 0:
            AP_2[i] = 0
            recal_2[i]=0
            f1_2[i]=0
        else:
            AP_2[i] = sum(np.array(P_2) * np.array(buffer_yes_2)) / sum(buffer_yes_2)
            recal_2[i]=AP_2[i]*sum(buffer_yes_2)/400
            f1_2[i]=2*AP_2[i]*recal_2[i]/(AP_2[i]+recal_2[i])
        if sum(buffer_yes_3) == 0:
            AP_3[i] = 0
            recal_2[i]=0
            f1_2[i]=0
        else:
            AP_3[i] = sum(np.array(P_3) * np.array(buffer_yes_3)) / sum(buffer_yes_3)
            recal_3[i]=AP_3[i]*sum(buffer_yes_3)/400
            f1_3[i]=2*AP_3[i]*recal_3[i]/(AP_3[i]+recal_3[i])

        # 排序后的汉明距离矩阵

        order_map_1 = [similarity_1[i] for i in y2_1] ## contain values of hamming distance in ascending order
        # print("order_map_1", order_map_1)
        order_map_2 = [similarity_2[i] for i in y2_2]
        order_map_3 = [similarity_3[i] for i in y2_3]

        pic_path1 = {}
        pic_path2 = {}
        pic_path3 = {}

        for k, v in enumerate(y2_1, 0):
            # print("k",k)
            # print("v",v)
            pic_path1[v] = k  ## value- position of the similarity value  key-- index value

        # for step,k in enumerate(y2_1):
        #     print("step",step)
        #     print("k",k)
        #     pic_path1[k] = step
        for k, v in enumerate(y2_2, 0):
            pic_path2[v] = k
        for k, v in enumerate(y2_3, 0):
            pic_path3[v] = k

        hanming_normalazition1 = hanming_normalazition(order_map_1)

        hanming_normalazition2 = hanming_normalazition(order_map_2)
        hanming_normalazition3 = hanming_normalazition(order_map_3)

        # print("hanming_normalazition1", hanming_normalazition1)

        # 汉明距离归一化 * 精度
        hanming_matrix1 = [i * acc[0] for i in hanming_normalazition1]
        hanming_matrix2 = [i * acc[1] for i in hanming_normalazition2]
        hanming_matrix3 = [i * acc[2] for i in hanming_normalazition3]

        path, new_list = list_add(K, pic_path1, pic_path2, pic_path3, hanming_matrix1, hanming_matrix2, hanming_matrix3)

        t1 = time.time()

        dictionary = {}
        for step, p in enumerate(path):
            value = dictionary.get(p, [])
            value.append(step)
            dictionary[p] = value
        # print("dictionary",dictionary)
        path_arr = [None for _ in range(len(path))]
        num_arr = [0 for _ in range(len(path))]

        for k, v in dictionary.items():
            path_arr[v[0]] = k
            num_arr[v[0]] = new_list[v[0]]

        final_path = [i for i in path_arr if i is not None][:K]
        final_path_dic = {}
        for step, r in enumerate(final_path):
            final_path_dic[step] = r
        # print("final_path_dic",final_path_dic)
        final_num = [i for i in num_arr if i][:K]

        final_index = argsort(final_num)
        final_index_dict = {}
        for step, r in enumerate(final_index):
            final_index_dict[step] = r

        lable_final = fn(train_label[0], train_base_dic, final_path_dic, final_index_dict)

#         print("lable_final", lable_final)
        count = 0.0
        sum_value = 0.0
        for step, r in enumerate(lable_final[:top_k]):
            step += 1
#             print(r)
            if str(query_label_3) == str(r):
                count += 1
                sum_value += count / step
        if count :
#             print('@@@@@')
            ap = sum_value / count
        else:
            ap = 0
        ap1 += ap
        recal+= ap*count /(400)
        F1score+= 2*ap*recal/(ap+recal)
#         print("ap", ap)
#         print("AP_1[i]", AP_1[i])
#         print("AP_2[i]", AP_2[i])
#         print("AP_3[i]", AP_3[i])
        t2 = time.time()
        #print("AP时间", t2 - t1)

    map_1 = np.mean(AP_1)
    map_2 = np.mean(AP_2)
    map_3 = np.mean(AP_3)
    print("recall for VGG")
    print(np.mean(recal_1))
    print("recall for Resnet")
    
    print(np.mean(recal_2))
    print("recall for Densenet")
    
    print(np.mean(recal_3))
#     print("F1score for VGG")
    
#     print(np.mean(f1_1))
#     print("F1score for Resnet")
    
#     print(np.mean(f1_2))
#     print("F1score for Densenet")
    
#     print(np.mean(f1_3))
    ap1 = ap1 / QueryTimes
    recal=recal/QueryTimes
    F1score=F1score/QueryTimes
    print(recal,"recall for ensemble model")
#     print(F1score,"F1score for ensemble model")
    # return precision_at_k, map
    return map_1, map_2, map_3, ap1


if __name__ == '__main__':
    # vgg resnet dense
    acc = [0.669, 0.692, 0.705]
    # 测试集
    # F:\project\produce_hash\NPY
    # F:\project\produce_hash\MAP_集成.py
#     traint_binary_densenet=traint_binary_resnet
#     traint_densenet=traint_resnet
#     testt_binary_densenet=testt_binary_resnet
#     testt_densenet=testt_resnet
    
    
    TestData_VGG = traintt_binary_vgg
    TestLabel_VGG = traint_vgg
    TestPath_VGG = traint_vgg
    # 训练集
    DataBase_VGG = testtt_binary_vgg[:1024]
    DataLabel_VGG = testt_vgg[:1024]
    DataPath_VGG = testt_vgg[:1024]

    # 测试集
    TestData_Resnet = traintt_binary_resnet
    TestLabel_Resnet = traint_resnet
    TestPath_Resnet = traint_resnet
    # 训练集
    DataBase_Resnet = testtt_binary_resnet[:1024]
    DataLabel_Resnet = testt_resnet[:1024]
    DataPath_Resnet = testt_resnet[:1024]

    # 测试集
    TestData_Desnet = traintt_binary_densenet
    TestLabel_Desnet = traint_densenet
    TestPath_Desnet = traint_densenet
    # 训练集
    DataBase_Desnet = testtt_binary_densenet[:1024]
    DataLabel_Desnet = testt_densenet[:1024]
    DataPath_Desnet = testt_densenet[:1024]

    DataLabel = [DataLabel_VGG, DataLabel_Resnet, DataLabel_Desnet]
    DataBase = [DataBase_VGG, DataBase_Resnet, DataBase_Desnet]
    DataPath = [DataPath_VGG, DataPath_Resnet, DataPath_Desnet]

    TestLabel = [TestLabel_VGG, TestLabel_Resnet, TestLabel_Desnet]
    TestData = [TestData_VGG, TestData_Resnet, TestData_Desnet]
    TestPath = [TestPath_VGG, TestPath_Resnet, TestPath_Desnet]
    print(len(TestLabel[0]))
    map = precision( TestLabel, TestData, TestPath,DataLabel, DataBase, DataPath, 200)

    filewrite = open("map_sun397_48bit_top1000754_bit.txt", "w")
    for i in map:
        filewrite.write("map: " + str(i) + "\n")
        filewrite.write("bit_num: " + "48\n")
        print(i)


187808
896


 90%|████████████████████████████████████████████████████████████████████████████▏        | 803/896 [1:32:33<10:09,  6.56s/it]